In [1]:
import numpy as np
import pandas as pd
import sys
sys.path.insert(1, '/Users/chengpeng/Desktop/Research/STAT/CIBer')
import comonotonic as cm
import os
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
import copy
import utils
import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTE
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler

In [2]:
df = pd.read_excel("HSI.xlsx")
df = df.drop(df.columns[[0]], axis=1)
colnames = [('X'+str(i)) for i in range(df.shape[1]-1)]
colnames.append('Y')
df.columns = colnames
scaler = preprocessing.MinMaxScaler()
df[colnames[:-1]] = scaler.fit_transform(df[colnames[:-1]])

In [4]:
cont_col = [i for i in range(7)]
df_train, df_test = train_test_split(df, test_size=0.3, stratify=df[['Y']])
scaler = preprocessing.StandardScaler()
scale_col_name = ["X"+str(i) for i in cont_col]
df_train[scale_col_name] = scaler.fit_transform(df_train[scale_col_name])
reduced_df_train = utils.outlier_removal(df_train, cont_col)
df_test[scale_col_name] = scaler.transform(df_test[scale_col_name])

<ipython-input-4-4d6404655c82>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train[scale_col_name] = scaler.fit_transform(df_train[scale_col_name])
/Users/chengpeng/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1736: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value[:, i].tolist())
<ipython-input-4-4d6404655c82>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See t

In [5]:
x_train = reduced_df_train.iloc[:,:-1].to_numpy()
y_train = reduced_df_train.iloc[:,-1].to_numpy()
smote = SMOTE(random_state=0)
x_train_synthetic, y_train_synthetic = smote.fit_resample(x_train, y_train)
x_test = df_test.iloc[:,:-1].to_numpy()
y_test = df_test.iloc[:,-1].to_numpy()

In [7]:
ciber = cm.clustered_comonotonic(x_train_synthetic,y_train_synthetic,None,cont_col,
                                      [], 0.8, None, corrtype='spearman',
                                      discrete_method='mdlp')
ciber.run()
ciber_predict = c_como_demo.predict(x_test)
print(classification_report(y_test, ciber_predict))

              precision    recall  f1-score   support

           0       0.98      0.90      0.94       154
           1       0.43      0.80      0.56        15

    accuracy                           0.89       169
   macro avg       0.70      0.85      0.75       169
weighted avg       0.93      0.89      0.90       169

